# Labo 8 : modèles de langage / Cloze Test
Par Guidoux Vincent et Gonzalez Montes Nathan

## Objectif et plan

L'objectif de ce labo est d'entraîner des modèles de langues (EN : language models) sous NLTK en utilisant le package nltk.lm. Les modèles seront entraînés sur des romans de Balzac en français
(fournis par le projet Gutenberg), et leurs performances seront mesurées par leur perplexité sur de nouveaux textes. Les modèles serviront également à deviner des mots cachés dans un texte, et ici leurs performances seront comparées à celles des humains sur cette même tâche.

## Importation

In [1]:
import nltk
import nltk.lm
from nltk.lm.preprocessing import padded_everygram_pipeline
import os, codecs
from urllib import request
import random

nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Vincent
[nltk_data]     Guidoux\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Obtenir les données

Télécharger les dix romans de la Comédie humaine de Balzac en français, enlever les notices en anglais au début et à la fin, puis les assembler en un corpus.

In [2]:
def dl_gutenberg_raw(url):
    print("dowloading \"{}\" ... ".format(url))
    
    response = request.urlopen(url)
    raw = response.read().decode('utf8')
    
    # enlever les notices en anglais au début 
    begin = raw.find("Libraries)") + 10
    # et à la fin
    end = raw.find("*** END OF THIS PROJECT")
    
    return raw[begin:end].lower()

In [3]:
volume_1 = "https://www.gutenberg.org/ebooks/41211.txt.utf-8"
volume_2 = "https://www.gutenberg.org/files/43851/43851-0.txt"
volume_3 = "https://www.gutenberg.org/files/45060/45060-0.txt"
volume_4 = "https://www.gutenberg.org/files/48082/48082-0.txt"
volume_5 = "https://www.gutenberg.org/files/49482/49482-0.txt"
volume_6 = "https://www.gutenberg.org/files/51381/51381-0.txt"
volume_7 = "https://www.gutenberg.org/files/52831/52831-0.txt"
volume_8 = "https://www.gutenberg.org/files/54723/54723-0.txt"
volume_9 = "https://www.gutenberg.org/files/55860/55860-0.txt"
volume_10 = "https://www.gutenberg.org/files/58244/58244-0.txt"

balzac = [volume_1, 
          volume_2, 
          volume_3, 
          volume_4, 
          volume_5, 
          volume_6, 
          volume_7, 
          volume_8,
          volume_9, 
          volume_10]

Utiliser une procédure à laquelle on donnera la liste de noms de fichiers ou des URLs.

In [4]:
corpus = ""

for volume_url in balzac:
    corpus += dl_gutenberg_raw(volume_url)

dowloading "https://www.gutenberg.org/ebooks/41211.txt.utf-8" ... 
dowloading "https://www.gutenberg.org/files/43851/43851-0.txt" ... 
dowloading "https://www.gutenberg.org/files/45060/45060-0.txt" ... 
dowloading "https://www.gutenberg.org/files/48082/48082-0.txt" ... 
dowloading "https://www.gutenberg.org/files/49482/49482-0.txt" ... 
dowloading "https://www.gutenberg.org/files/51381/51381-0.txt" ... 
dowloading "https://www.gutenberg.org/files/52831/52831-0.txt" ... 
dowloading "https://www.gutenberg.org/files/54723/54723-0.txt" ... 
dowloading "https://www.gutenberg.org/files/55860/55860-0.txt" ... 
dowloading "https://www.gutenberg.org/files/58244/58244-0.txt" ... 


In [5]:
print("Il y a {} caractères dans ce corpus.".format(len(corpus)))

Il y a 12214531 caractères dans ce corpus.


Sauvegarder le texte résultant : quelle taille fait-il en Ko ou Mo?

In [6]:
filename1 = "la_comedie_humaine.txt"

if os.path.exists(filename1): 
    os.remove(filename1)
fd = codecs.open(filename1, 'a', 'utf8')

try:
    fd.write(corpus)
    

finally:  
    fd.close()

Il fait **12'258 [ko]** et **12 [Mo]**

### Segmenter et tokenizer en une liste de listes de mots, y compris les ponctuations (une liste = une phrase).

In [7]:
corpus_nettoye = corpus

In [8]:
# corpus_nettoye = corpus_nettoye.replace('.\r\n',' ') # on nettoie les retours à la ligne
# corpus_nettoye = corpus_nettoye.replace('\r\n.',' ') # on nettoie les retours à la ligne
# corpus_nettoye = corpus_nettoye.replace('\r\n',' ') # on nettoie les retours à la ligne
corpus_nettoye = corpus_nettoye.replace('_',' ') # on nettoie les _

In [9]:
tokenized = nltk.sent_tokenize(corpus_nettoye, language="french")

In [10]:
print("Il y a {} phrases dans le corpus".format(len(tokenized)))

Il y a 101622 phrases dans le corpus


In [11]:
tokenized = [nltk.word_tokenize(sentence) for sentence in tokenized]

In [12]:
word_nb = 0
for sentence in tokenized:
    word_nb += len(sentence)

In [13]:
print("Il y a {} mots dans ce corpus".format(word_nb))

Il y a 2335507 mots dans ce corpus


In [14]:
tokenized[87:93]

[['balzac',
  ',',
  'de',
  'peur',
  "qu'ils",
  'ne',
  "s'ébruitassent",
  ',',
  'les',
  'changeait',
  'souvent',
  '.'],
 ['nous',
  'nous',
  'souvenons',
  'de',
  'ceux-ci',
  ':',
  'au',
  'portier',
  "l'on",
  'disait',
  ':',
  '«',
  'la',
  'saison',
  'des',
  'prunes',
  'est',
  'arrivée',
  ',',
  '»',
  'et',
  'il',
  'vous',
  'laissait',
  'franchir',
  'le',
  'seuil',
  ';',
  'au',
  'domestique',
  'accouru',
  'sur',
  "l'escalier",
  'au',
  'son',
  'de',
  'la',
  'cloche',
  ',',
  'il',
  'fallait',
  'murmurer',
  ':',
  '«',
  "j'apporte",
  'des',
  'dentelles',
  'de',
  'belgique',
  ',',
  '»',
  'et',
  'si',
  'vous',
  'assuriez',
  'au',
  'valet',
  'de',
  'chambre',
  'que',
  '«',
  'madame',
  'bertrand',
  'était',
  'en',
  'bonne',
  'santé',
  ',',
  '»',
  'on',
  'vous',
  'introduisait',
  'enfin',
  '.'],
 ['ces',
  'enfantillages',
  'amusaient',
  'beaucoup',
  'balzac',
  ';',
  'ils',
  'étaient',
  'peut-être',
  'nécessai

In [15]:
tokenized[1506:1510]

[['lorsque',
  'le',
  'chef',
  'de',
  'la',
  'famille',
  'éprouva',
  'quelque',
  'refroidissement',
  'dans',
  'la',
  'tacite',
  'et',
  'précaire',
  'amitié',
  'du',
  'monarque',
  ',',
  'il',
  'trembla',
  "d'autant",
  'plus',
  ',',
  'que',
  ',',
  'par',
  'suite',
  'des',
  'défis',
  'railleurs',
  'de',
  'ses',
  'soeurs',
  ',',
  'jamais',
  'sa',
  'fille',
  'chérie',
  "n'avait",
  'jeté',
  'ses',
  'vues',
  'si',
  'haut',
  '.'],
 ['au',
  'milieu',
  'de',
  'ces',
  'circonstances',
  'et',
  'au',
  'moment',
  'où',
  'cette',
  'petite',
  'lutte',
  'domestique',
  'était',
  'devenue',
  'fort',
  'grave',
  ',',
  'le',
  'monarque',
  ',',
  'auprès',
  'duquel',
  'monsieur',
  'de',
  'fontaine',
  'croyait',
  'rentrer',
  'en',
  'grâce',
  ',',
  'fut',
  'attaqué',
  'de',
  'la',
  'maladie',
  'dont',
  'il',
  'devait',
  'périr',
  '.'],
 ['le',
  'grand',
  'politique',
  'qui',
  'sut',
  'si',
  'bien',
  'conduire',
  'sa',
  '

In [16]:
tokenized[7000:7005]

[['après',
  'le',
  'dîner',
  ',',
  'adam',
  'les',
  'laissa',
  'seuls',
  ',',
  'et',
  'clémentine',
  'gronda',
  'thaddée',
  'de',
  'manière',
  'à',
  'lui',
  'faire',
  'sentir',
  "qu'elle",
  'ne',
  'le',
  'voulait',
  'plus',
  'au',
  'logis',
  '.'],
 ['--',
  'oui',
  ',',
  'madame',
  ',',
  'dit',
  'humblement',
  'thaddée',
  ',',
  'vous',
  'avez',
  'raison',
  ',',
  'je',
  'suis',
  'un',
  'misérable',
  ',',
  "j'avais",
  'donné',
  'ma',
  'parole',
  '.'],
 ['mais', 'que', 'voulez-vous', '?'],
 ["j'avais",
  'remis',
  'à',
  'quitter',
  'malaga',
  'après',
  'le',
  'carnaval',
  '...',
  'je',
  'serai',
  'franc',
  ',',
  "d'ailleurs",
  ':',
  'cette',
  'femme',
  'exerce',
  'un',
  'tel',
  'empire',
  'sur',
  'moi',
  'que',
  '...',
  '--',
  'une',
  'femme',
  'qui',
  'se',
  'fait',
  'mettre',
  'à',
  'la',
  'porte',
  'de',
  'chez',
  'musard',
  'par',
  'les',
  'sergents',
  'de',
  'ville',
  ',',
  'et',
  'pour',
  'qu

Extraire un fragment (environ 2000 mots) qui servira de donnée de test.

In [17]:
def separate_train_test(sentences):
    
    train_percentage = 0.03

    random.shuffle(sentences)
    
    sentences_length = len(sentences)
    
    train_set_length = int(len(sentences) * train_percentage)
    
    test_set = sentences[ :train_set_length]
    
    train_set = sentences[train_set_length: ]

    return (train_set, test_set)

In [18]:
train_set, test_set = separate_train_test(tokenized)

In [19]:
print("Il y a {} phrases dans le train_set et {} dans le test_set".format(
        len(train_set),
        len(test_set)))

Il y a 98574 phrases dans le train_set et 3048 dans le test_set


## Entraîner un premier modèle de langage de NLTK

En utilisant les instructions disponibles pour le module NLTK LM, entraîner un modèle de langage sur les données d'entraînement. Attention, le package lm n'est disponible qu'à partir de NLTK version 3.4.

In [20]:
from nltk.lm import MLE
import time

Commencer avec un modèle MLE à l'ordre 2, comme montré dans le tutoriel.

In [21]:
order = 5

In [22]:
train, vocab = padded_everygram_pipeline(order, tokenized)
#train, vocab = padded_everygram_pipeline(order, train_set)

In [23]:
lm = MLE(order)

In [24]:
begin = time.time()
lm.fit(train, vocab)
end = time.time()

print("Cela a pris {:.3f} secondes pour entraîner le modèle d'ordre {}".format(
            (end-begin), order))

Cela a pris 98.203 secondes pour entraîner le modèle d'ordre 5


Calculer la perplexité du modèle sur l’ensemble de test.

In [25]:
lm.perplexity(test_set)

inf

Générer quelques phrases dans le style de Balzac selon les explications de NLTK.

In [26]:
def print_generated_sentence(length, sentence_nb):

    for i in range(sentence_nb):
        sentence = "{}. ".format(i)
        for word in lm.generate(length):
            if word != "<s>" and word != "</s>":
                sentence += word + " "
        print(sentence + "\n")

In [27]:
print_generated_sentence(16, 2)

0. la première et se mit auprès d'elle pour la séparer du jeune vicomte . 

1. un bourbier . 



In [28]:
print_generated_sentence(25, 50)

0. jour de sa réhabilitation . 

1. vie . 

2. , fluet , et ne l'avance pas ; ses yeux , où les scintillements des pierres semblaient se répéter , brillaient d'un feu surnaturel ; 

3. 

4. dans l'omnipotence de leurs modes , dans la diversité de leurs mélancolies , avec les teintes de leurs méditatives extases , avec les jets impétueux 

5. par le licou de la lettre de calyste , elle fut traversée par la pointe d'une idée cruelle . 

6. 

7. 

8. 

9. les déchirera . 

10. qui vers midi fut constaté . 

11. soucient plus guère . 

12. 

13. 

14. coudes sur le dossier du fauteuil occupé par le vieux gentilhomme en imitant , sans le savoir , surpassait en éclat le lever sidéral des 

15. 

16. 

17. cette nomination froissa beaucoup le juge desfondrilles , toujours archéologue et plus que jamais suppléant . 

18. 

19. extravagantes dont les bougies jaunes ne s'allumaient qu'aux jours où la présidente dépouillait de son enveloppe verte un vieux lustre à pendeloques de cristal de 

20. 

## Entraîner un second modèle de langage de NLTK

En utilisant par exemple le modèle de Laplace.

Le tester aussi sur le corpus de test et comparer les scores.

## Cloze test

Supprimer un mot sur 7 dans le corpus de test.

En utilisant la méthode lm.generate avec du contexte, demander au modèle de langage de prédire ces mots (en utilisant donc 2-4 mots précédents). Quelle est la performance du système ? Comparer les modèles générés en (2) et en (3).

Générez un texte avec les mots en question (1 mot sur 7) remplacés par des blancs, et passez-le à un autre binôme : quelle est leur performance pour deviner les mots ?

Merci d’envoyer votre notebook Jupyter par email au professeur avant le vendredi 14 juin à 23h59.